## RAFT 학습용 Bad documents 선별 작업


In [ ]:
import sys
import os

# 현재 스크립트의 디렉토리 기준으로 상위 폴더 경로 추가
sys.path.append(os.path.abspath("../"))

In [3]:
import numpy as np
import pandas as pd
import pinecone
from tqdm import tqdm

from utils import get_pinecone_index

In [ ]:
# Pinecone index 연결
pinecone_index, index_name = get_pinecone_index()

Connected to Pinecone index: sling-chunk


In [ ]:
# bad_docs 추가할 CSV 파일 로드
input_file = "data/train_v2.0.1.csv"  # 원본 CSV 파일 경로
output_file = "data/train_v2.0.1_with_bad_docs.csv"  # 결과 저장 경로
data = pd.read_csv(input_file)

In [ ]:
# 임의 벡터 생성 함수
def generate_random_vector(dim=1024):
    """512차원의 랜덤 벡터 생성"""
    return np.random.rand(dim).tolist()

In [ ]:
# 결과 저장 리스트
bad_docs = []

# tqdm을 사용해 진행 상황 표시
for _, row in tqdm(data.iterrows(), total=len(data), desc="Processing Bad Documents"):
    try:
        # 랜덤 벡터 생성
        random_vector = generate_random_vector()

        # Pinecone에서 쿼리 실행
        results = pinecone_index.query(
            vector=random_vector, top_k=1, include_metadata=True  # 랜덤 벡터로 검색  # 1개의 문서만 가져오기
        )

        # 검색 결과 처리
        if results and "matches" in results and len(results["matches"]) > 0:
            bad_doc = results["matches"][0]["metadata"]["text"]  # 첫 번째 문서 텍스트
        else:
            bad_doc = "No document found"  # 검색 결과가 없을 경우 기본값
    except Exception as e:
        print(f"Error processing row {row.get('id', 'unknown')}: {e}")
        bad_doc = "Error occurred while retrieving document"

    # 결과 리스트에 추가
    bad_docs.append(bad_doc)

Processing Bad Documents: 100%|██████████| 1627/1627 [05:36<00:00,  4.83it/s]


In [ ]:
bad_docs[0]

'영국과 프랑스가 합의에 나섰지만, 덴마크가 공국의 분할에 끝까지 반대하여 협상은 실패하였고, 결국 10월 빈 조약이 체결되어 두 공국은 오스트리아와 프로이센에 넘겨졌다. 이후 가슈타인 협정에 따라 슐레스비히 공국은 프로이센이, 홀슈타인 공국은 오스트리아가 차지하게 된다. 또한 홀슈타인 남부의 라우엔부르크 공국은 프로이센령이 되었다. 이로 인하여 프로이센의 월경지가 오스트리아 내부에 존재하게 되는 문제가 생겨났으며 프로이센이 주도하는 북독일에 한정한 독일 통일에도 오스트리아가 걸림돌이 되어버렸다.'

In [ ]:
# bad_docs 리스트를 새로운 열로 추가
data["bad_docs2"] = bad_docs

In [ ]:
data.head()

,id,paragraph,question,choices,answer,question_plus,bad_docs,bad_docs2
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"['ㄱ, ㄴ', 'ㄱ, ㄷ', 'ㄴ, ㄹ', 'ㄷ, ㄹ']",2,ㄱ.기 사환국으로 정권을 장악하였다 .ㄴ.인 조반정을 주도 하여 집권세력이 되었다 ...,사학연금\n사립학교교직원연금공단으로부터 받은 ‘사학 퇴직연금 수급자 현황’ 자료에 ...,"영국과 프랑스가 합의에 나섰지만, 덴마크가 공국의 분할에 끝까지 반대하여 협상은 실..."
1,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,(가) 지역에 대한 설명으로 옳은 것은?,"['이곳에 대장도감을 설치하여 재조대장경을 만들었다.', '지눌이 이곳에서 수선사 ...",4,NaN,「임차료지급보증」\n월세임대차계약을 체결한 임차인이 임대인에 지급하여야 할 월 임대...,인도\n인도 국민의 약 68%가 종사하는 농업은 자급적 성격이 강한 인도의 가장 중...
2,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,밑줄 친 ‘그’에 대한 설명으로 옳은 것은?,"['살수에서 수의 군대를 물리쳤다 .', '김춘추 의 신라 왕위 계승을 지원하였다 ...",2,NaN,종류\n아이스크림 콘(Ice cream Cone): 아이스크림을 콘에 담은 것이다....,나카무라는 미국에서 연구를 계속하기 위해 형편상 미국 시민권을 취득했는데 노벨상 수...
3,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",(가) 인물이 추진한 정책으로 옳지 않은 것은?,"['사창제를 실시하였다 .', '대전회통을 편찬하였다 .', '비변사의 기능을 강화...",3,NaN,연혁\n연표 (도쿄 대학 창설 후)\n1877년 4월 - 도쿄 대학 창설.\n187...,위하여 필수적인 주거공간을 마련하기 위한 것이라면 일상의 가사에 속한다고 볼 여지가...
4,generation-for-nlp-431,(가)의 사신 고제덕 등이 일본에 와서 왕이 보낸 국서를 전하였다. 그 국서에 이르...,(가) 국가에 대한 설명으로 옳은 것은?,"['나 당연합군의 공격으로 멸망하였다 .', '9주 5소경의 지방 행정 구역을 두었...",3,NaN,"곤충의 생식계는 생식선(生殖腺)과 생식관, 부속샘[腺]들로 이루어져 있다.\n곤충의...",at high temperature and high pressure to cause...


In [40]:
data.to_csv(output_file)